In [0]:
%pip install nutter

  Obtaining dependency information for nutter from https://files.pythonhosted.org/packages/33/18/b73b699a249a6ddbd53f80ff5293c08f4e4357f835a6a0682e7f6f1a54c8/nutter-0.1.35-py3-none-any.whl.metadata
  Using cached nutter-0.1.35-py3-none-any.whl.metadata (636 bytes)
  Obtaining dependency information for databricks-api from https://files.pythonhosted.org/packages/d0/cc/6c3f9cd8b2b6c7a45c95b94d334bc51f1579d875bbfac0ecb8accdb2f756/databricks_api-0.9.0-py3-none-any.whl.metadata
  Using cached databricks_api-0.9.0-py3-none-any.whl.metadata (17 kB)
  Using cached fire-0.7.0-py3-none-any.whl
  Obtaining dependency information for junit-xml from https://files.pythonhosted.org/packages/2a/93/2d896b5fd3d79b4cadd8882c06650e66d003f465c9d12c488d92853dff78/junit_xml-1.9-py2.py3-none-any.whl.metadata
  Using cached junit_xml-1.9-py2.py3-none-any.whl.metadata (3.2 kB)
  Obtaining dependency information for py4j from https://files.pythonhosted.org/packages/53/75/15967ccc1a9bb2c85364a4eceb64116fbf8734528

In [0]:
dbutils.library.restartPython()

In [0]:
import logging
import time
import functools
from typing import List
from pyspark.sql import DataFrame
from pyspark.sql.functions import (
    current_timestamp, to_date, date_format, col, 
    trim, encode, lit, regexp_replace, when
)
from pyspark.sql.types import StringType, VarcharType
from pyspark.ml import Pipeline
from pyspark.ml.base import Transformer
from pyspark.ml.param.shared import HasInputCols, HasOutputCols
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable

In [0]:
# Basic logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [0]:
def timer_logger(func):
    """Simple decorator to log function execution time"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        logging.info(f"Starting {func.__name__}")
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        logging.info(f"Finished {func.__name__} in {execution_time:.2f} seconds")
        return result
    return wrapper

## Unit testing

**Ensuring :**

**No Nulls in data**

**Price consistency (Ex: highprice>lowprice)**

**Schema Validation**

**All measurements are positive**

**Time values are in correct format (HH:MM:SS)**

**row_count > 100000**

In [0]:
from runtime.nutterfixture import NutterFixture
from pyspark.sql.functions import col, sum, lit
from pyspark.sql.types import TimestampType, DoubleType, IntegerType, StringType, DateType

class UnitTestingStockData(NutterFixture):
    def __init__(self):
        super().__init__()
        # Load DataFrame from silver table in our database
        self.df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/bd_finals.db/stocks/silver/stocks_silver")

    @timer_logger
    def assertion_row_count(self):
        """Test that the dataset has rows"""
        row_count = self.df.count()
        assert row_count > 100000, f"Expected at least 100000 row, but got {row_count}"
    
    @timer_logger
    def assertion_no_nulls(self):
        """Test that there are no null values in any column"""
        # Get count of nulls for each column
        null_counts = self.df.select([
            sum(col(c).isNull().cast("int")).alias(c) 
            for c in self.df.columns
        ]).first()
        
        # Check each column for nulls
        for column in self.df.columns:
            null_count = null_counts[column]
            assert null_count == 0, f"Found {null_count} null values in column {column}"

    def assertion_schema_validation(self):
        """Test for presence and data types of required columns"""
        expected_columns = {
            "date": DateType(),
            "time": StringType(),
            "company": StringType(),
            "open_price": DoubleType(),
            "high_price": DoubleType(),
            "low_price": DoubleType(),
            "close_price": DoubleType(),
            "volume": DoubleType(),
            "number_of_trades": IntegerType()
        }
        
        for col_name, col_type in expected_columns.items():
            assert col_name in self.df.columns, f"Column {col_name} is missing"
            actual_type = self.df.schema[col_name].dataType
            assert isinstance(actual_type, type(col_type)), \
                f"Column {col_name} has incorrect type. Expected {col_type}, got {actual_type}"
    @timer_logger
    def assertion_price_consistency(self):
        """Test price relationships (high >= open/close >= low)"""
        inconsistent_prices = self.df.filter(
            (col("high_price") < col("low_price")) |
            (col("high_price") < col("open_price")) |
            (col("high_price") < col("close_price")) |
            (col("low_price") > col("open_price")) |
            (col("low_price") > col("close_price"))
        ).count()
        
        assert inconsistent_prices == 0, \
            f"Found {inconsistent_prices} rows with inconsistent price relationships"
    @timer_logger
    def assertion_positive_values(self):
        """Test that volume, number_of_trades, and prices are positive"""
        negative_values = self.df.filter(
            (col("volume") <= 0) |
            (col("number_of_trades") <= 0) |
            (col("open_price") <= 0) |
            (col("high_price") <= 0) |
            (col("low_price") <= 0) |
            (col("close_price") <= 0)
        ).count()
        
        assert negative_values == 0, \
            f"Found {negative_values} rows with negative or zero values"
    @timer_logger
    def assertion_time_format(self):
        """Test that time values are in correct format (HH:MM:SS)"""
        # Convert time string to timestamp to validate format
        time_format_errors = self.df.filter(
            ~col("time").rlike("^([0-1][0-9]|2[0-3]):[0-5][0-9]:[0-5][0-9]$")
        ).count()
        
        assert time_format_errors == 0, \
            f"Found {time_format_errors} rows with invalid time format"


2024-12-19 23:30:25,318 - INFO - Received command c on object id p0


In [0]:
# Execute the test fixture
result = UnitTestingStockData().execute_tests()
print(result.to_string())

2024-12-19 23:30:30,639 - INFO - Starting assertion_no_nulls
2024-12-19 23:30:36,088 - INFO - Finished assertion_no_nulls in 5.45 seconds
2024-12-19 23:30:36,088 - INFO - Starting assertion_positive_values
2024-12-19 23:30:36,758 - INFO - Finished assertion_positive_values in 0.67 seconds
2024-12-19 23:30:36,759 - INFO - Starting assertion_price_consistency
2024-12-19 23:30:39,843 - INFO - Finished assertion_price_consistency in 3.08 seconds
2024-12-19 23:30:39,845 - INFO - Starting assertion_row_count
2024-12-19 23:30:41,083 - INFO - Finished assertion_row_count in 1.24 seconds
2024-12-19 23:30:41,084 - INFO - Starting assertion_time_format
2024-12-19 23:30:44,468 - INFO - Finished assertion_time_format in 3.38 seconds



Notebook: N/A - Lifecycle State: N/A, Result: N/A
Run Page URL: N/A
PASSING TESTS
------------------------------------------------------------
no_nulls (5.448992083998746 seconds)
positive_values (0.6706039860000601 seconds)
price_consistency (3.085743506999279 seconds)
row_count (1.2388307400033227 seconds)
schema_validation (7.695199747104198e-05 seconds)
time_format (3.3856531649944372 seconds)





## Gold Tables & Analysis

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

2024-12-19 20:33:27,608 - INFO - Received command c on object id p0


In [0]:
df1 = (
    spark.read.format("delta")
    .option("header", True)
    .option("inferSchema", True)
    .load("dbfs:/user/hive/warehouse/bd_finals.db/stocks/silver/stocks_silver")
)

2024-12-19 19:52:11,101 - INFO - Received command c on object id p0


## Apple Stock Prediction Over the years

provides insights into Apple's stock performance trends over time and generates predictions for future years based on historical data.

In [0]:
def predict_apple_stock(df1):
    # Filter only Apple data and create yearly aggregation
    apple_yearly = (
        df1.filter(col("company") == "AAPL")
        .withColumn("year", year(to_date(col("date"))))
        .groupBy("year")
        .agg(
            avg("close_price").alias("avg_price"),
            avg("volume").alias("avg_volume"),
            count("*").alias("data_points"),
        )
        .orderBy("year")
    )
    # Creating time index for prediction
    window_spec = Window.orderBy("year")
    apple_data = apple_yearly.withColumn("time_index", row_number().over(window_spec))

    # Preparing features
    assembler = VectorAssembler(inputCols=["time_index"], outputCol="features")
    data = assembler.transform(apple_data)

    # Training Linear Regression model
    lr = LinearRegression(featuresCol="features", labelCol="avg_price")
    model = lr.fit(data)

    # Making predictions
    predictions = model.transform(data)

    # Print performance metrics
    print("\nApple Stock Prediction Metrics:")
    print(f"R² Score: {model.summary.r2:.4f}")
    print(f"Root Mean Squared Error: ${model.summary.rootMeanSquaredError:.2f}")

    # Display yearly predictions with year-over-year change
    result = predictions.select(
        "year",
        round("avg_price", 2).alias("actual_price"),
        round("prediction", 2).alias("predicted_price"),
    )
    # Add year-over-year price change
    result = result.withColumn(
        "yoy_change",
        round(
            (
                (col("actual_price") - lag("actual_price").over(window_spec))
                / lag("actual_price").over(window_spec)
                * 100
            ),
            2,
        ),
    )
    print("\nApple Yearly Stock Analysis:")
    result.display()
    return result

In [0]:
#saving the table in gold folder
t1=predict_apple_stock(df1)
t1.coalesce(1).write.mode('overwrite') \
    .option('header', 'true') \
    .csv("dbfs:/user/hive/warehouse/bd_finals.db/stocks/gold/apple_stock_predictions.csv")

2024-12-19 21:37:09,823 - INFO - Received command c on object id p0



Apple Stock Prediction Metrics:
R² Score: 0.9963
Root Mean Squared Error: $1.78

Apple Yearly Stock Analysis:


year,actual_price,predicted_price,yoy_change
2022,132.23,133.49,null
2023,171.83,169.31,29.95
2024,203.88,205.14,18.65


Databricks visualization. Run in Databricks to view.

## return percentage for each company

This code calculates the price change and return percentage for each company in the dataset (df1) using a window specification that partitions the data by company and orders it by date and time.

Return Percentage: Calculates the percentage change between the last and first prices relative to the first price.

In [0]:
# Calculating price change and return percentage for each company
window_spec = Window.partitionBy("company").orderBy("date", "time")
price_analysis = (
    df1.withColumn(
        "first_price", first("close_price").over(Window.partitionBy("company"))
    )
    .withColumn("last_price", last("close_price").over(Window.partitionBy("company")))
    .withColumn("price_change", round(col("last_price") - col("first_price"), 2))
    .withColumn(
        "return_percentage",
        round((col("last_price") - col("first_price")) / col("first_price") * 100, 2),
    )
    .select("company", "first_price", "last_price", "price_change", "return_percentage")
    .distinct()
    .orderBy(desc("return_percentage"))
)

print("Stock Performance Analysis (Ordered by Return Percentage):")
price_analysis.display()
# Saving as single CSV file in gold folder
price_analysis.coalesce(1).write.mode('overwrite') \
    .option('header', 'true') \
    .csv("dbfs:/user/hive/warehouse/bd_finals.db/stocks/gold/company_returns.csv")

Stock Performance Analysis (Ordered by Return Percentage):


company,first_price,last_price,price_change,return_percentage
NVDA,17.212,142.49,125.28,727.85
META,115.69,619.45,503.76,435.44
SHOP,36.64,113.665,77.03,210.22
AVGO,56.279,171.81,115.53,205.28
NFLX,314.4,917.907,603.51,191.96
CRM,132.99,354.48,221.49,166.55
AMZN,87.825,225.3329,137.51,156.57
UBER,26.51,64.6,38.09,143.68
ORCL,80.311,177.64,97.33,121.19
TSLA,156.315,339.14,182.83,116.96


Databricks visualization. Run in Databricks to view.

## Total trading volume by company

This code calculates the total trading volume and the average number of trades for each company in the dataset (df1) by grouping on the company column.

In [0]:
total_volume = df1.groupBy('company') \
    .agg(
        sum('volume').alias('total_volume'),
        round(avg('number_of_trades'), 0).alias('avg_trades')
    ) \
    .orderBy(desc('total_volume'))

print(" Trading Volume Summary by Company:")
total_volume.display()
# Saving as single CSV file in gold folder
price_analysis.coalesce(1).write.mode('overwrite') \
    .option('header', 'true') \
    .csv("dbfs:/user/hive/warehouse/bd_finals.db/stocks/gold/total_volume.csv")

 Trading Volume Summary by Company:


company,total_volume,avg_trades
NVDA,1.67883379697E11,1237.0
TSLA,5.0503622297E10,1394.0
AMD,2.7431445897E10,605.0
AAPL,2.4849202233E10,835.0
INTC,2.1392828723E10,359.0
AMZN,1.9326253393E10,635.0
GOOGL,1.2693047954E10,542.0
SNAP,1.1051224812E10,164.0
AVGO,1.0870060105E10,266.0
MSFT,9.813544374E9,614.0


Databricks visualization. Run in Databricks to view.